In [ ]:
using JLD2, FileIO

In [ ]:
using Plots
gr(html_output_format=:png)



In [ ]:
include("../src/TreeAerodynamics.jl")


In [ ]:
D0 = 0.8

η1 = [0.0, 0.1, 0.25, 0.40, 0.55, 0.7, 0.849, 1.0, 1.15, 1.30, 1.45, 1.6, 1.8, 2, 2.5, 3, 3.5, 4, 5];
x1 = TA.geomseq(0.0, 100*D0, 30, 1.12); 

#wakes = [TA.wake2d(wm, x1, η1) for i in 1:nb];

In [ ]:
Cd = TA.DragCoeff(1.18, 0.32, 0.2)
branch = TA.Branch2d(D0, Cd, 0.0, 0.0)
branches2 = TA.HexagonTree(branch);
branches3 = TA.HexagonTree(branches2);
branches4 = TA.HexagonTree(branches3);
branches5 = TA.HexagonTree(branches4);
uoofun1(x,y) = (10.0, 0.0)
#Uoo = 20.0:-2.0:2.0
Uoo = [40; 30; 25; 22; 20:-2.0:2]

In [ ]:
function simulatevels(branches, Uoo, x, η; err=1e-5, rlx=0.2)
    
    nu = length(Uoo)
    nb = length(branches)
    drag = []
    
    println("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    println("Vel: ", Uoo[1])
    push!(drag, TA.velinterference(branches, (x,y)->(Uoo[1], 0.0), x, η; err=err, rlx=rlx))
    
    for i = 2:nu
    println("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    println("Vel: ", Uoo[i])
        fun = (x,y) -> (Uoo[i], 0.0)
        Ux = drag[i-1][1] .* Uoo[i]/Uoo[i-1]
        Uy = drag[i-1][1] .* Uoo[i]/Uoo[i-1]
        rlx = min(0.2, rlx*1.5)
        push!(drag, TA.velinterference(branches, fun, x, η, err=err, rlx=rlx, Uxinit=Ux, Uyinit=Uy))
    end
    
    return drag
    
    
end

In [ ]:
drag2 = simulatevels(branches2.branches, Uoo, x1, η1; err=1e-5, rlx=0.1)

In [ ]:
drag3 = simulatevels(branches3.branches, Uoo, x1, η1; err=1e-4,rlx=0.1)

In [ ]:
drag4 = simulatevels(branches4.branches, Uoo, x1, η1; err=1e-4, rlx=0.03)

In [ ]:
@save "circle2.jld2" Cd, branch, branches2, branches3, drag2, drag3

In [ ]:
Ux2 = hcat([d[1] for d in drag2]...)
Uy2 = hcat([d[2] for d in drag2]...)
Cd2 = hcat([d[3] for d in drag2]...)
xc2 = [b.xc for b in branches2.branches]
yc2 = [b.yc for b in branches2.branches]        
D2 = TA.diameter(branches2)

Ux3 = hcat([d[1] for d in drag3]...)
Uy3 = hcat([d[2] for d in drag3]...)
Cd3 = hcat([d[3] for d in drag3]...)
xc3 = [b.xc for b in branches3.branches]
yc3 = [b.yc for b in branches3.branches];
D3 = TA.diameter(branches3)


In [ ]:
maximum(abs, Uy3b - Uy3[:,5:end])

In [ ]:
scatter(xc2, yc2, aspect_ratio=1.0, color=:green)
ii = 1
quiver!(xc2, yc2, gradient=(Ux2[:,ii]./2Uoo[ii], Uy2[:,ii]./2Uoo[ii]), color=:red)
ii = 14
quiver!(xc2, yc2, gradient=(Ux2[:,ii]./2Uoo[ii], Uy2[:,ii]./2Uoo[ii]), color=:blue)


In [ ]:
scatter(xc3, yc3, aspect_ratio=1.0, color=:green)
ii = 1
quiver!(xc3, yc3, gradient=(2Ux3[:,ii]./Uoo[ii], 2Uy3[:,ii]./2Uoo[ii]), color=:red)
ii = 10
quiver!(xc3, yc3, gradient=(2Ux3[:,ii]./Uoo[ii], 2Uy3[:,ii]./2Uoo[ii]), color=:blue)


In [ ]:
function forcetot(branches, Ux, Uy, ρ=1.2)

    Fx = 0.0
    Fy = 0.0
    nb = length(branches)
    for i = 1:nb
        ux = Ux[i]
        uy = Uy[i]
        U = hypot(ux, uy)
        Cd = TA.dragcoeff(branches[i], U)
        D = TA.diameter(branches[i])
        α = atan(uy, ux)
        cα = cos(α)
        sα = sin(α)
        fi = 0.5 * ρ * Cd * D * U^2
        
        Fx += fi*cα
        Fy += fi*sα
    end

    return Fx, Fy
end

function force(branch, ux, uy, ρ=1.2)

    U = hypot(ux, uy)
    Cd = TA.dragcoeff(branch, U)
    D = TA.diameter(branch)
    α = atan(uy, ux)
    cα = cos(α)
    sα = sin(α)
    fi = 0.5 * ρ * Cd * D * U^2
        
    Fx = fi*cα
    Fy = fi*sα
    

    return Fx, Fy
end


function forcecoef(branches, Ux, Uy, Uoo, Lref)
    ρ = 1.0
    Fx, Fy = forcetot(branches, Ux, Uy, 1.0)
    CFx = Fx / (0.5 * ρ * Lref * Uoo^2)
    CFy = Fy / (0.5 * ρ * Lref * Uoo^2)    
    
    return CFx, CFy
end



In [ ]:
D1 = TA.diameter(branch)
D2 = TA.diameter(branches2)
D3 = TA.diameter(branches3)
CFx1 = [force(branch, uoo, 0.0, 1.0)[1]/(0.5 * D1 * uoo^2) for uoo in Uoo]
CFx2 = [forcecoef(branches2.branches , Ux2[:,i], Uy2[:,i], Uoo[i], D2)[1] for i in 1:length(Uoo)]
CFx3 = [forcecoef(branches3.branches , Ux3[:,i], Uy3[:,i], Uoo[i], D3)[1] for i in 1:length(Uoo)]
CFx = [CFx1 CFx2 CFx3]
d = [D1, D2, D3]

In [ ]:
rr = copy(CFx)
for i = 1:size(rr, 1)
    rr[i,:] = rr[i,:] / CFx[i,1]
end
#plot(d, CFx[1,:]/CFx[1,1], xscale=:log10, markershape=:circle, color=1)
plot(d, rr', xscale=:log10, markershape=:circle)
#plot!(d, CFx[2,:]/CFx[2,1], markershape=:circle, color=2)
#for i in 2:10 
#    plot!(p, d, CFx[i,:]/CFx[i,1], markershape=:circle, color=i)
#end
#show(p)
#plot!(d, CFx[2,:] marker = (markershape=:circle, color=:))


In [ ]:
CFx[1,:], d

In [ ]:
forcetot(branches2.branches , Ux2[:,1], Uy2[:,1], 1.2)[1] / (0.5 * 1.2 * 20^2 * 2.4)

In [ ]:
force(branch, 20, 0, 1.2)[1] * 7 / (0.5 * 1.2 * 20^2 * 2.4)

In [ ]:
collect(20.0:2.0:2.0)

In [ ]:
TA.dragcoeff(branch, 20) * 1.2/2 * TA.diameter(branch) * 20^2